##### install libaries w/ muted output

In [1]:

import subprocess

def pip_install(package_name):
    command = ["pip", "install", "-U", package_name]
    with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE) as process:
        process.wait()
        if process.returncode == 0:
            print("installed " + package_name)
        else: 
            raise Exception("Failed to install package: {}".format(package_name))
 
packages_to_install = ["openai", "langchain"]
for package_name in packages_to_install:
    pip_install(package_name)

installed openai
installed langchain


##### versioning audit

In [2]:
import sys
print("Python ver:", sys.version)

import openai
print("openai ver:", openai.__version__)

import langchain
print("langchain ver:", langchain.__version__)

import json
import os

Python ver: 3.11.5 (main, Aug 28 2023, 00:00:00) [GCC 13.2.1 20230728 (Red Hat 13.2.1-1)]
openai ver: 0.28.1
langchain ver: 0.0.314


##### import API key

In [3]:
def read_openai_api_key():
    # file folder location
    file_folder = "/location/of/your/file/folder"

    # API file name
    file_name = "OPENAI_API_KEY.json"

    # file path
    file_path = os.path.join(file_folder, file_name)

    # Check if API file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    # Check if API file is readable
    if not os.access(file_path, os.R_OK):
        raise PermissionError(f"Permission denied: {file_path}")

    # Open API file & read key
    with open(file_path, "r") as f:
        api_key = json.load(f)["OPENAI_API_KEY"]
      
    return api_key

openai_api_key = read_openai_api_key()

##### import LC functions

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt

##### read Langchain few shot JSON template and JSON inputs

In [22]:
title_prompt = load_prompt("title_zero_shot_prompt.json")

with open('zero_shot_prompt_input.json', 'r') as f:
    input_data = json.load(f)

##### call chat model & chain

In [23]:
# setup LC chat model
chat = langchain.chat_models.openai.ChatOpenAI(  
    openai_api_key=openai_api_key,
    model="gpt-3.5-turbo",
    max_tokens=50,
    temperature=0.05
)

chain = title_prompt | chat

result = chain.invoke(input_data)

##### convert chat result to JSON & save file

In [24]:
# convert
json_conv_input = result.content
json_conv_output = json.dumps({"title": json_conv_input})
print(json_conv_output)

# write to file
with open('zero_shot_prompt_output.json', 'w') as f:
    json.dump(json.loads(json_conv_output), f)

{"title": "Milwaukee Road Vintage Logo Tee | Orange"}
